<a href="https://colab.research.google.com/github/mirrormouse/machine_learning/blob/main/english_words_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

アルファベットと数字の変換準備

In [ ]:
char="abcdefghijklmnopqrstuvwxyz"
dic={}
for i in range(len(char)):
  dic[char[i]]=i
print(dic['l'])

11


ドライブをインポート

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


でたらめな５文字からなる「ランダム文字列」を生成。これを上のcharに対応させて数値に変換
例）["abcdz",...]->[[0,1,2,3,25],...]

In [ ]:
import random
def generate(data_size=1000,length=5):
  random_list=[]
  random_words=[]
  for i in range(data_size):
    word=''
    num=[]
    for j in range(length):
      a=random.randrange(26)
      c=char[a]
      word+=c
      num.append(a)
    random_list.append(num)
    random_words.append(word)
  return random_list,random_words

lists,word=generate()

英文データの読み込み

In [ ]:
base='drive/MyDrive/ML/train/'
f = open(base+'train.txt', 'r')
rawdata = f.read()
data=rawdata.split()
print(data[:5])
f.close()

['The', 'two', 'submarines', 'of', 'the']


５文字の長さの英単語だけを大量に抽出し、「存在する英単語」としてまとめる。これも数値に変換。

In [ ]:
import re
def correct_generate(rawdata,data_size=10000,length=5):
  alpha_data=[]
  for word in rawdata:
    word=word.lower()
    res = re.sub(r'[^a-zA-Z]', '', word)
    if len(res)==length:
      alpha_data.append(res)
  set_alpha=list(set(alpha_data))

  beta_data=[]
  for word in set_alpha:
    num=[]
    for id in range(len(word)):
      c=word[id]
      num.append(dic[c])
    beta_data.append(num)
  size=len(beta_data)
  print(size)
  return set_alpha, beta_data, size

_,beta,_=correct_generate(data)
print(beta[:5])

961
[[2, 0, 19, 2, 7], [3, 17, 14, 22, 13], [2, 11, 14, 18, 4], [22, 0, 19, 2, 7], [1, 17, 8, 13, 6]]


訓練データの生成（上で作った２つを繋げるだけ）

In [ ]:
x_train_rawlist=lists+beta
y_train_list=[]
for i in range(len(lists)):
  y_train_list.append(0)
for i in range(len(beta)):
  y_train_list.append(1)
print(x_train_rawlist[0])

[11, 1, 22, 16, 10]


訓練データのリストをnumpyに変換。正解ラベルを生成

In [ ]:
import numpy as np
x_train_list=[]
for data in x_train_rawlist:
  item=[]
  for num in data:
    one_hot=np.zeros(26)
    one_hot[num]=1
    one_hot_list=one_hot.tolist()
    item.append(one_hot_list)
  x_train_list.append(item)
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
print(x_train[0])
print(y_train[-5:])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
[1 1 1 1 1]


In [ ]:
print(x_train.shape)
print(y_train.shape)

(1961, 5, 26)
(1961,)


ディープラーニングモデルを作る

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
model_1 = tf.keras.Sequential(name='my_model')
model_1.add(tf.keras.layers.Flatten(input_shape=(5,26), name='flatten_layer_1'))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(256, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(512, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(256, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.1))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.1))
model_1.add(tf.keras.layers.Dense(2, activation='softmax'))

model_1.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_layer_1 (Flatten)    (None, 130)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 128)               16768     
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_32 (Dropout)         (None, 512)               0  

モデルのコンパイル

In [ ]:
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

学習！

In [ ]:
history = model_1.fit(x_train, y_train, batch_size=128, epochs=40,
                    validation_split=0.1,shuffle=True )

Epoch 1/40
14/14 [==============================] - 1s 26ms/step - loss: 0.6508 - accuracy: 0.5737 - val_loss: 0.6018 - val_accuracy: 0.7716
Epoch 2/40
14/14 [==============================] - 0s 15ms/step - loss: 0.4178 - accuracy: 0.8282 - val_loss: 0.2213 - val_accuracy: 0.9340
Epoch 3/40
14/14 [==============================] - 0s 16ms/step - loss: 0.2845 - accuracy: 0.8810 - val_loss: 0.3820 - val_accuracy: 0.8680
Epoch 4/40
14/14 [==============================] - 0s 15ms/step - loss: 0.2266 - accuracy: 0.9167 - val_loss: 0.2369 - val_accuracy: 0.9239
Epoch 5/40
14/14 [==============================] - 0s 15ms/step - loss: 0.1838 - accuracy: 0.9286 - val_loss: 0.2018 - val_accuracy: 0.9289
Epoch 6/40
14/14 [==============================] - 0s 14ms/step - loss: 0.1578 - accuracy: 0.9450 - val_loss: 0.3543 - val_accuracy: 0.8782
Epoch 7/40
14/14 [==============================] - 0s 15ms/step - loss: 0.1413 - accuracy: 0.9501 - val_loss: 0.3748 - val_accuracy: 0.8782
Epoch 8/40
14

In [ ]:
def nothing_val(word):#元データ内にその単語が存在するか確認
  for w in alpha:
    if word==w:
      return False
  return True
nothing_val('joker')
nothing_val('beast')
nothing_val('criry')

True

単語とベクトルの変換用関数

In [ ]:
def word2num(word):
  num=[]
  for id in range(len(word)):
    num.append(dic[word[id]])
  return num
def num2word(nums):
  word=''
  for i in nums:
    word+=char[i]
  return word
print(num2word([1,4, 0, 18, 19]))
print(word2num("beast"))

beast
[1, 4, 0, 18, 19]


テスト　訓練データに含めていない"beast"や"blade"を正しく「存在する英単語」と判断していることが分かる

In [ ]:
test_words=["beast","blade","joker","kdjus","adjsa"]
y_test=[1,1,1,0,0]
x_test=[]
for word in test_words:
  vec=word2num(word)
  item=[]
  for num in vec:
    data=np.zeros(26,dtype=int)
    data[num]=1
    data_list=data.tolist()
    item.append(data_list)
  x_test.append(item)
print(x_test)
test_loss, test_acc = model_1.evaluate(x_test, y_test, verbose=0)
print(test_loss)
print(test_acc)
predictions = model_1.predict(x_test)
print(predictions)

[[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

大量のランダム文字列とペクトルを生成し、予測結果の「存在する単語」である確率が大きい順にソート

In [ ]:
length=5
randomlist,randomword=generate(data_size=100000)
input=[]
for vec in randomlist:
  item=[]
  for num in vec:
    data=np.zeros(26,dtype=int)
    data[num]=1
    data_list=data.tolist()
    item.append(data_list)
  input.append(item)

r_pred=model_1.predict(input)
item=[]
for i in range(len(r_pred)):
  res=[i,r_pred[i][0]]
  item.append(res)
item.sort(key=lambda item:item[1])

予測結果の「存在する単語」である確率が最も大きい30単語を表示。
なんとなく「英単語にありそう」な雰囲気の文字列があることがわかる（？）

In [ ]:
top=30
ans=[]
for i in range(top):
  id=item[i][0]
  ans.append([randomword[id],item[i][1]])
  print([randomword[id],item[i][1]])

['aresn', 8.140479e-12]
['fross', 9.360529e-11]
['srirm', 1.4889086e-10]
['siesd', 2.8570077e-10]
['suand', 3.077406e-10]
['sramn', 3.6880232e-10]
['slosn', 3.9755757e-10]
['coias', 6.4331496e-10]
['bresn', 6.9942746e-10]
['spise', 7.33037e-10]
['suyse', 9.483382e-10]
['sxaae', 9.978939e-10]
['aposd', 1.1661512e-09]
['sroks', 1.2433329e-09]
['auese', 1.3839621e-09]
['sracd', 1.4601056e-09]
['srund', 1.9346504e-09]
['crobs', 2.1252802e-09]
['siiil', 2.2484026e-09]
['aliad', 3.0197276e-09]
['siind', 3.1254523e-09]
['sioas', 3.6240808e-09]
['arasc', 3.9477634e-09]
['aoasm', 5.764397e-09]
['criry', 6.511828e-09]
['srimk', 6.735973e-09]
['adind', 6.998508e-09]
['shabt', 7.3702595e-09]
['wuory', 7.948066e-09]
['seacl', 8.938689e-09]
